In [ ]:
!nvidia-smi

Sun Dec 18 12:19:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pytorch-fid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytorch-fid: filename=pytorch_fid-0.2.1-py3-none-any.whl size=14834 sha256=f28f7e38d68264efda682e57433a7063710f5b4cf21a2dfd5c8a47ac6a07805f
  Stored in directory: /root/.cache/pip/wheels/df/c8/a0/cce2ed7671ae52be132ae836e429bba6148544f83b7962b4bc
Successfully built pytorch-fid


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Style transfer'

/content/drive/MyDrive/Style transfer


In [ ]:
%pwd

'/content/drive/MyDrive/Style transfer'

In [ ]:
# !git clone https://github.com/gordicaleksa/pytorch-neural-style-transfer.git

In [ ]:
%cd 'pytorch-neural-style-transfer'

/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer


In [ ]:
%ls

data/            neural_style_transfer.py
environment.yml  README.md
LICENCE          reconstruct_image_from_representation.py
models/          utils/


In [ ]:
import utils.utils as utils
from utils.video_utils import create_video_from_intermediate_results

import torch
from torch.optim import Adam, LBFGS
from torch.autograd import Variable
import numpy as np
import os
import argparse

default_resource_dir = '/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data'
content_images_dir = os.path.join(default_resource_dir, 'content-images')
style_images_dir = os.path.join(default_resource_dir, 'style-images')
output_img_dir = os.path.join(default_resource_dir, 'output-images')
img_format = (4, '.jpg')  # saves images in the format: %04d.jpg

optimization_config = dict()
optimization_config['height'] = 400
optimization_config['content_weight'] = 1e7
optimization_config['style_weight'] = 3e4 # 2 * 10^4 init <- 4 x 1o^4 
optimization_config['tv_weight'] = 1e0
optimization_config['optimizer'] = 'lbfgs'
optimization_config['model'] = 'vgg19'
optimization_config['init_method'] = 'content'
optimization_config['saving_freq'] = -1

# init config : (cw : 1e7, sw : 4e4, tw : 1e0)
# new config : (cw : 1e7, sw : 5e4, tw : 1e-1)

optimization_config['content_images_dir'] = content_images_dir
optimization_config['style_images_dir'] = style_images_dir
optimization_config['output_img_dir'] = output_img_dir
optimization_config['img_format'] = img_format

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

neural_net, content_feature_maps_index_name, style_feature_maps_indices_names = utils.prepare_model('vgg19', device)

def build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    target_content_representation = target_representations[0]
    target_style_representation = target_representations[1]

    current_set_of_feature_maps = neural_net(optimizing_img)

    current_content_representation = current_set_of_feature_maps[content_feature_maps_index].squeeze(axis=0)
    content_loss = torch.nn.MSELoss(reduction='mean')(target_content_representation, current_content_representation)

    style_loss = 0.0
    current_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(current_set_of_feature_maps) if cnt in style_feature_maps_indices]
    for gram_gt, gram_hat in zip(target_style_representation, current_style_representation):
        style_loss += torch.nn.MSELoss(reduction='sum')(gram_gt[0], gram_hat[0])
    style_loss /= len(target_style_representation)

    tv_loss = utils.total_variation(optimizing_img)

    total_loss = config['content_weight'] * content_loss + config['style_weight'] * style_loss + config['tv_weight'] * tv_loss

    return total_loss, content_loss, style_loss, tv_loss


def make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    # Builds function that performs a step in the tuning loop
    def tuning_step(optimizing_img):
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config)
        # Computes gradients
        total_loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        return total_loss, content_loss, style_loss, tv_loss

    # Returns the function that will be called inside the tuning loop
    return tuning_step


def neural_style_transfer(config, content_img_path, style_img_path):
    # content_img_path = os.path.join(config['content_images_dir'], config['content_img_name'])
    # style_img_path = os.path.join(config['style_images_dir'], config['style_img_name'])

    # out_dir_name = '/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data/Generated-images'
    dump_path = config['output_img_dir']
    os.makedirs(dump_path, exist_ok=True)

    content_img = utils.prepare_img(content_img_path, config['height'], device)
    style_img = utils.prepare_img(style_img_path, config['height'], device)

    if config['init_method'] == 'random':
        # white_noise_img = np.random.uniform(-90., 90., content_img.shape).astype(np.float32)
        gaussian_noise_img = np.random.normal(loc=0, scale=90., size=content_img.shape).astype(np.float32)
        init_img = torch.from_numpy(gaussian_noise_img).float().to(device)
    elif config['init_method'] == 'content':
        init_img = content_img
    else:
        # init image has same dimension as content image - this is a hard constraint
        # feature maps need to be of same size for content image and init image
        style_img_resized = utils.prepare_img(style_img_path, np.asarray(content_img.shape[2:]), device)
        init_img = style_img_resized

    # we are tuning optimizing_img's pixels! (that's why requires_grad=True)
    optimizing_img = Variable(init_img, requires_grad=True)

    content_img_set_of_feature_maps = neural_net(content_img)
    style_img_set_of_feature_maps = neural_net(style_img)

    target_content_representation = content_img_set_of_feature_maps[content_feature_maps_index_name[0]].squeeze(axis=0)
    target_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(style_img_set_of_feature_maps) if cnt in style_feature_maps_indices_names[0]]
    target_representations = [target_content_representation, target_style_representation]

    # magic numbers in general are a big no no - some things in this code are left like this by design to avoid clutter
    num_of_iterations = {
        "lbfgs": 1000,
        "adam": 3000,
    }

    #
    # Start of optimization procedure
    #
    if config['optimizer'] == 'adam':
        optimizer = Adam((optimizing_img,), lr=1e1)
        tuning_step = make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
        for cnt in range(num_of_iterations[config['optimizer']]):
            total_loss, content_loss, style_loss, tv_loss = tuning_step(optimizing_img)
            with torch.no_grad():
                print(f'Adam | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
                utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, num_of_iterations[config['optimizer']], should_display=False)
    elif config['optimizer'] == 'lbfgs':
        # line_search_fn does not seem to have significant impact on result
        optimizer = LBFGS((optimizing_img,), max_iter=num_of_iterations['lbfgs'], line_search_fn='strong_wolfe')
        cnt = 0

        def closure():
            nonlocal cnt
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
            if total_loss.requires_grad:
                total_loss.backward()
            with torch.no_grad():
                if(cnt%200 == 0):
                  print(f'L-BFGS | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
                utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, num_of_iterations[config['optimizer']], should_display=False, image_name = os.path.split(content_img_path)[1].split('.')[0] + '_' + os.path.split(style_img_path)[1].split('.')[0])

            cnt += 1
            return total_loss

        optimizer.step(closure)

    return dump_path

    # uncomment this if you want to create a video from images dumped during the optimization procedure
    # create_video_from_intermediate_results(results_path, img_format)

## **Final Style Transfer**

In [ ]:
np.random.seed(2020)

### **ship 1 and ship 2**

In [ ]:
# content_img_list = ['/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data/content-images/67.png']
ship1_path = "/content/drive/MyDrive/Style transfer/generated-images-models/ship1"
ship2_path = "/content/drive/MyDrive/Style transfer/generated-images-models/ship2"

file_list_1 = []

for root, dirs, files in os.walk(ship1_path):
    for name in files:
      file_list_1.append(os.path.join(root, name))

for root, dirs, files in os.walk(ship2_path):
    for name in files:
      file_list_1.append(os.path.join(root, name))

In [ ]:
file_list_1[-10:]

['/content/drive/MyDrive/Style transfer/generated-images-models/ship2/17.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/6.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/3.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/2.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/4.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/5.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/0.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/1.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/7.png',
 '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/8.png']

In [ ]:
n = 10
content_img_list_first = np.random.choice(file_list_1, n, replace = False)

In [ ]:
content_img_list_first[:10]

array(['/content/drive/MyDrive/Style transfer/generated-images-models/ship1/24.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/28.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/74.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship1/65.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship1/122.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/132.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/2.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/44.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/133.png',
       '/content/drive/MyDrive/Style transfer/generated-images-models/ship2/31.png'],
      dtype='<U82')

In [ ]:
import time

style_path = '/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data/style-images/ship-150.png'
output_img_dir = os.path.join(default_resource_dir, 'ship1_2_new')
optimization_config['style_img_name'] = os.path.basename(style_path)
optimization_config['output_img_dir'] = output_img_dir

t1 = time.time()

for content_path in content_img_list_first:
  print(f'Processing file : {os.path.split(content_path)[1]}')
  optimization_config['content_img_name'] = os.path.basename(content_path)
  results_path = neural_style_transfer(optimization_config, content_path, style_path)
  print()

t2 = time.time()

Processing file : 24.png
L-BFGS | iteration: 000, total loss=1871196061696.0000, content_loss=      0.0000, style loss=1871193840000.0000, tv loss=2220095.0000
L-BFGS | iteration: 200, total loss=94369497088.0000, content_loss=77596752929.6875, style loss=16745150625.0000, tv loss=27592228.0000
L-BFGS | iteration: 400, total loss=88129937408.0000, content_loss=73913378906.2500, style loss=14185954687.5000, tv loss=30608566.0000
L-BFGS | iteration: 600, total loss=86829654016.0000, content_loss=72953291015.6250, style loss=13842929062.5000, tv loss=33431948.0000
L-BFGS | iteration: 800, total loss=86311272448.0000, content_loss=72568876953.1250, style loss=13706739375.0000, tv loss=35661724.0000
L-BFGS | iteration: 1000, total loss=85996847104.0000, content_loss=72332338867.1875, style loss=13626879375.0000, tv loss=37636208.0000

Processing file : 28.png
L-BFGS | iteration: 000, total loss=2168155144192.0000, content_loss=      0.0000, style loss=2168153520000.0000, tv loss=1521166.000

KeyboardInterrupt: ignored

In [ ]:
print(f'Total time taken for processing {n} files : {t2 - t1}s')

### **ship 3, 4 and 5**

In [ ]:
# content_img_list = ['/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data/content-images/67.png']
ship3_path = "/content/drive/MyDrive/Style transfer/generated-images-models/ship3"
ship4_path = "/content/drive/MyDrive/Style transfer/generated-images-models/ship4"
ship5_path = "/content/drive/MyDrive/Style transfer/generated-images-models/ship5"

file_list_2 = []

for root, dirs, files in os.walk(ship3_path):
    for name in files:
      file_list_2.append(os.path.join(root, name))

for root, dirs, files in os.walk(ship4_path):
    for name in files:
      file_list_2.append(os.path.join(root, name))

for root, dirs, files in os.walk(ship5_path):
    for name in files:
      file_list_2.append(os.path.join(root, name))

In [ ]:
file_list_2[-10:]

In [ ]:
n = 50
content_img_list_second = np.random.choice(file_list_2, n, replace = False)

In [ ]:
content_img_list_second[:10]

In [ ]:
import time

style_path = '/content/drive/MyDrive/Style transfer/pytorch-neural-style-transfer/data/style-images/ship-030.png'
output_img_dir = os.path.join(default_resource_dir, 'ship3_4_5_new')
optimization_config['style_img_name'] = os.path.basename(style_path)
optimization_config['output_img_dir'] = output_img_dir

t1 = time.time()

for content_path in content_img_list_second:
  print(f'Processing file : {os.path.split(content_path)[1]}')
  optimization_config['content_img_name'] = os.path.basename(content_path)
  results_path = neural_style_transfer(optimization_config, content_path, style_path)
  print()

t2 = time.time()

In [ ]:
print(f'Total time taken for processing {n} files : {t2 - t1}s')

## **Evaluation**

In [ ]:
%ls

data/            neural_style_transfer.py
environment.yml  README.md
LICENCE          reconstruct_image_from_representation.py
models/          utils/


In [ ]:
%cd /content/drive/MyDrive/Style transfer

/content/drive/.shortcut-targets-by-id/1kQzhYFmZ6JrdPsG4tM0vxDYVfuEAcIKV/Style transfer


In [ ]:
# !python -m pytorch_fid Generated-images  plane-real

In [ ]:
# ! git clone https://github.com/mseitzer/pytorch-fid.git

fatal: destination path 'pytorch-fid' already exists and is not an empty directory.


In [ ]:
# FID score between two different generated images using different stype images
! python pytorch-fid/src/pytorch_fid/fid_score.py ./pytorch-neural-style-transfer/data/ship1_2 ./pytorch-neural-style-transfer/data/ship3_4_5

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:11<00:00, 8.15MB/s]
100% 2/2 [00:21<00:00, 10.84s/it]
100% 3/3 [00:08<00:00,  2.99s/it]
FID:  203.75461757391366


In [ ]:
# FID Score comparision between real sonar images of planes and ships 
! python pytorch-fid/src/pytorch_fid/fid_score.py ./pytorch-neural-style-transfer/data/plane-real ./pytorch-neural-style-transfer/data/plane-real-2

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100% 7/7 [00:25<00:00,  3.66s/it]
100% 3/3 [00:05<00:00,  1.87s/it]
FID:  170.179870602696


In [ ]:
# FID Score compirision between two sets of real sonar ship images ? 

In [ ]:
# FID score comparision between ship1_2
! python pytorch-fid/src/pytorch_fid/fid_score.py ./pytorch-neural-style-transfer/data/plane-real ./pytorch-neural-style-transfer/data/ship1_2_new

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100% 7/7 [00:04<00:00,  1.52it/s]
100% 1/1 [00:16<00:00, 16.61s/it]
FID:  363.1913822436733


In [ ]:
# FID score comparision between ship3_4_5
! python pytorch-fid/src/pytorch_fid/fid_score.py ./pytorch-neural-style-transfer/data/plane-real ./pytorch-neural-style-transfer/data/ship3_4_5

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100% 7/7 [00:04<00:00,  1.50it/s]
100% 3/3 [00:02<00:00,  1.32it/s]
FID:  313.8197164999362
